# main.py explained

The included `main.py` is a sample script that programmatically controls the OSC1Lite stimulation controller using Python. This is a detailed document of `main.py`. This document assumes you have basic knowledge of Python.

This script requires the OpalKelly Python interface. The OpalKelly Python interface is a SWIG wrapper of the C API. Unfortunately the python dll is statically linked, so we must use the same version of Python interpreter as the one included in SDK. The version included in windows SDK is Python 3.5.

In [ ]:
#! /usr/bin/env python3.5

"""
This is a sample script that programmatically controls OSC1Lite using the API
"""


To use the API, you need to import the OpalKelly SDK and the osc1lite file. If you have not installed the OpalKelly SDK to Python's package path, you need to manually copy the `ok.py` and `_ok.pyd` to the same folder.

In [ ]:
import ok        # The OpalKelly SDK, you may need to manually copy it to current folder
import osc1lite  # The OSC1Lite python interface

Set the logging level to either specified by `LOGLEVEL` environment variable, or `DEBUG`.

In [ ]:
import os

# Enable debug logging
import logging
logging.basicConfig(level=os.environ.get("LOGLEVEL", "DEBUG"))

Before creating the OSC1Lite object, we need to enumerate and connect to one OpalKelly board. See https://opalkelly.com/examples/enumerating-devices/#tab-python and https://opalkelly.com/examples/open-a-specific-device/#tab-python for detailed explanation.

We will use the `serial` variable later to locate the calibration file, so you must specify it here. Otherwise the board will run in uncalibrated mode, which has a low accuracy and possibly a zero-scale leakage.

In [ ]:
# Initialize OpalKelly
dev = ok.okCFrontPanel()

# Enumerate devices
n_devices = dev.GetDeviceCount()
for i in range(n_devices):
    logging.debug(
        'Device[{0}] Model: {1}'.format(i, dev.GetDeviceListModel(i)))
    logging.debug(
        'Device[{0}] Serial: {1}'.format(i, dev.GetDeviceListSerial(i)))
assert n_devices, 'No connected device. Check the connection and make sure no other program is occupying the device.'

# Open the default device
#serial = '1740000JJK'
serial = ''  # Fill in the serial of your board here!
dev.OpenBySerial(serial)
assert dev.IsOpen(), 'Device open failed. Is the FPGA dead?'

Now load the calibration data. All OSC1Lites are callibrated before shipping, and you can find the corresponding calibration file in the `calib/` folder.
The calibration file contains 12 rows, one row per channel. Each row has 3 numbers, they are:

* The voltage drop (V) across the limiting resistor, when the board is in uncalibrated mode and amplitude is set to 10uA;
* The voltage drop (V) across the limiting resistor, when the board is in uncalibrated mode and amplitude is set to 90uA;
* The resistance (kOhm) of the limiting resistor.

There was an old calibration format which only has 2 columns, and the resistance is assumed to be 100 kOhm. Those boards are never shipped to customers.

The calib array accepted by OSC1Lite API, hoever, is a list of 2-element tuples. Each tuple corresponds to one channel. The 2 numbers are:

* The actual current (mA) across the limiting resistor, when the board is in uncalibrated mode and amplitude is set to 10uA;
* The actual current (mA) across the limiting resistor, when the board is in uncalibrated mode and amplitude is set to 90uA.

Therefore we need to divide first 2 elements by the 3rd element in each row.

See the REMARK below about the order of the channels.

In [ ]:
# Load the calibration data
try:
    with open('calib/' + serial + '.calib') as fp:
        calib = []
        for _ in range(12):
            s = next(fp).strip().split(None, 2)
            s[0] = float(s[0])
            s[1] = float(s[1])
            if len(s) == 3:
                s[2] = float(s[2])
                s[0] /= s[2]
                s[1] /= s[2]
            else:
                s[0] /= 100
                s[1] /= 100
            calib.append(s[0:2])
except:
    # just use dummy data
    calib = [None for _ in range(12)]

Now we can create the OSC1Lite object with the OpalKelly device and the calibration data.

In [ ]:
# Initialize OSC1Lite board
osc = osc1lite.OSC1Lite(dev, calib=calib)

Before using the OSC1Lite object, we reset the whole system.

* `osc.configure()` will reset the FPGA;
* `osc.reset()` will reset all communication protocols between FPGA and PC, FPGA and DACs;
* `osc.init_dac()` will reset the DACs, and write the calibration data to DACs;
* `osc.enable_dac_output()` will enable FPGA to send output commands to DACs.

In [ ]:
osc.configure(bit_file='OSC1_LITE_Control.bit', ignore_hash_error=False)
osc.reset()
osc.init_dac()
osc.enable_dac_output()

Enable all 12 DAC channels. Before enabling the channels, the output pin will stay floating after reset. After enabling the channel, the output will be connected to the DAC. So even if a channel is not used, enabling the channel may decrease unwanted noise.

You can also disable a channel by calling `osc.set_enable(channel, False)`. Note that enabling / disabling the channel will create a small voltage glitch, so it is not recommended to enable / disable channel during experiment.

Note: In OSCGUI, all channels are automatically enabled after connected to the board, and disabled before disconnecting from the board.

In [ ]:
# Enable all 12 channels
osc.set_enable(range(12), True)

Set all channels to continuous mode. `True` is for continuous mode, and `False` is for one-shot mode.

In [ ]:
# Set all channels to continuous mode
osc.set_trigger_mode(range(12), True)

Set all channels to PC trigger. `True` is for external, and `False` is for PC trigger.

Note: You can use continuous mode with external trigger when using API. The waveform will be reset on every trigger in rise edge.

In [ ]:
# Set all channels to PC trigger
osc.set_trigger_source(range(12), False)

Use `osc.set_channel()` to configure the waveform parameters of each channel. Here we set all channels to square wave with no rise time, 50uA amplitude, 0.1s pulse width and 0.2s period. 

The firse parameter is `mode` which is related to rise time. Refer to comment in `osc1lite.py` for the relationship between mode and rise time.

In [ ]:
# Configure the waveform parameters of each channel
for ch in range(12):
    osc.set_channel(ch, osc1lite.ChannelInfo(
        osc1lite.SquareWaveform(0, 50, .1, .2)))

Send PC trigger to all channels using `osc.trigger_channel()`. There is no effect if the channel is using external trigger.

In [ ]:
# Send PC trigger to all channels
osc.trigger_channel(range(12))

Wait for Enter key. The OSC1Lite will output the waveform as configured above.

In [ ]:
input('Now LED on all channels should be flashing. Press enter to exit')

Disable all channels and disconnect the OpalKelly device.

Note: if you do not explicitly disable all channels, they will keep the last state even if the board is disconnected. If you accidentally did this, you can re-connect to the board, disable all channels and disconnect again. Or you can use OSCGUI, and connect / disconnect to the board.

In [ ]:
# Disable channels
osc.set_enable(range(12), False)

# Disconnect the OpalKelly device
dev.Close()

## REMARK: the channel order

In the OSC1Lite API, all channels are specified with channel index. In OSCGUI, however, channels are in shank name. The mapping is:

| Channel Index | Shank Name |
|---------------|------------|
| 0             | S2L1       |
| 1             | S2L3       |
| 2             | S1L2       |
| 3             | S4L1       |
| 4             | S4L3       |
| 5             | S3L2       |
| 6             | S2L2       |
| 7             | S1L1       |
| 8             | S1L3       |
| 9             | S4L2       |
| 10            | S3L1       |
| 11            | S3L3       |